In [17]:
%%writefile mapper1.py


import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

path = 'stop_words_en.txt'

# Your code for reading stop words here
with open(path) as f1:
    t = f1.readlines()
    stop_words_template = list(word.strip() for word in t)
    

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    
    lower_words = map(lambda x:x.lower(), words)
    counts = Counter(lower_words)
    
    
    total_words = 0
    
    for word, word_counts in counts.items():        
        if word not in stop_words_template:
            total_words += word_counts
    for word, word_counts in counts.items():        
        if word not in stop_words_template:
            tf = float(word_counts) / total_words
            print '%s\t%d\t%s' % (word, 0, 'idf')
            print '%s\t%d\t%f' % (word, int(article_id), tf)
            
    
    
    
    

Overwriting mapper1.py


In [55]:
%%writefile reducer1.py
import math
import sys

current_key = None
article_count = 0

for line in sys.stdin:
    try:
        key, article_id, tf = line.strip().split('\t')
        article_id = int(article_id)
    except ValueError as e:
        continue
    if current_key != key:
        article_count = 0
        current_key = key
    if article_id == 0:
        article_count += 1
    else:
        idf = 1/math.log(1 + article_count)
        tf = float(tf)
        tf_idf = tf*idf
        print "%s_%d\t%f" % (current_key, article_id,tf_idf)

if current_key:
    idf = 1/math.log(1 + article_count)
    tf = float(tf)
    tf_idf = tf*idf
    print "%s_%d\t%f" % (current_key, article_id,tf_idf)

Overwriting reducer1.py


In [33]:
%%writefile reducer.py

import math
import sys

current_key = None
article_count = 0

for line in sys.stdin:
    try:
        key, article_id, tf = line.strip().split('\t')
        article_id = int(article_id)
  
    except ValueError as e:
        continue
    print "%s\t%d\t%s" % (key, article_id,tf)
    

Overwriting reducer.py


In [59]:
%%bash

OUT_DIR1="week6_assignment_1_1_"$(date +"%s%6N")
NUM_REDUCERS=2
LOGS="stderr_logs.txt"

# Stub code for your job

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar -D mapred.jab.name="Streaming Stop Words" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D stream.num.map.output.key.fields=2 \
    -files mapper1.py,reducer1.py,/datasets/stop_words_en.txt \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR1} \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner > /dev/null
    
#hdfs dfs -get ${OUT_DIR1}/part-00000 idf.txt
 #-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
 #-D mapreduce.partition.keycomparator.options=-k1,1nr \
 #-D mapreduce.partition.keypartitioner.options="-k1,1" \
 #-D stream.num.map.output.key.fields=2 \
 #-files reduce_side_mapper_slice.py \
 #-mapper 'python reduce_side_mapper_slice.py' \
 #-numReduceTasks 5 \
 #-input /data/telecommunication,/user/adral/geojson \
 #-output telecom-joins \
 #-partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner
 
hdfs dfs -cat ${OUT_DIR1}/part-00000 ${OUT_DIR1}/part-00001 | grep -w "labor_12" | cut -f2

0.000351
